In [2]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import shutil
from torch.nn import Softmax

%matplotlib inline

In [3]:
root = Path('../output/runs')

CLASSES = ['HTC-1-M7', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 'Motorola-Nexus-6', 'Motorola-X',
           'Samsung-Galaxy-Note3', 'Samsung-Galaxy-S4', 'Sony-NEX-7', 'iPhone-4s', 'iPhone-6']

In [4]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In [103]:
#runs = ['densenet_ismanip','resnet101_ismanip','resnet50_ismanip','densenet121_final','resnet101_final','resnet50_final']
#runs = ['resnet101_final','resnet50_final']
suf = '_best_tta'

images = [str(p) for p in Path('../data/test').glob('*.tif')]
all_preds = []
last_fnames = None
for run in runs:
    preds, fnames = pickle.load(open(str(root/run/('predict_test'+suf+'.pkl')),'rb'))
    preds, fnames = np.vstack(preds), np.array(fnames)
    assert last_fnames is None or np.all(last_fnames == fnames)
    last_fnames = fnames
    preds = softmax(preds)
    all_preds.append(preds)

avg_preds = np.dstack(all_preds).mean(axis=2)

# for i in range(100):
#     print(all_preds[0][i].argmax(),all_preds[1][i].argmax(),all_preds[2][i].argmax(),avg_preds[i].argmax())
#     print()

avg_preds = avg_preds.argmax(axis=1)
fname = [Path(p).parts[-1] for p in images]
cameras = [CLASSES[i] for i in avg_preds]

pd.DataFrame({'fname': fname, 'camera': cameras}, columns=['fname', 'camera']).to_csv('../output/runs/ensemble0/predict_test'+suf+'.csv', index=False)